In [1]:
import tensorflow as tf
import numpy as np
import pickle
from PIL import Image as Im
from IPython.display import Image

import matplotlib.pyplot as plt
%matplotlib inline

/home/letfoolsdie/virt_envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
train_data = unpickle("../data/processed_data/all_cf_batches.p")

with open("../data/processed_data/all_labels.txt") as lab:
    train_labels = [int(i) for i in lab.read().split()]
    
label_names = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [4]:
def to_categorical(data, num_classes):
    temp = np.zeros((len(data), num_classes))
    temp[range(len(data)), data] = 1
    return temp

train_labels = to_categorical(train_labels, 10)

In [5]:
train_data = train_data / 255

In [6]:
train_data, X_test, train_labels, Y_test = train_data[:45000], train_data[45000:], train_labels[:45000], train_labels[45000:]

In [7]:
X_train, X_val, Y_train, Y_val = train_data[:40000], train_data[40000:], train_labels[:40000], train_labels[40000:]

In [8]:
print("Train data:")
print(X_train.shape)
print("="*20)
print("Validation data:")
print(X_val.shape)
print("="*20)
print("Test data:")
print(X_test.shape)

Train data:
(40000, 32, 32, 3)
Validation data:
(5000, 32, 32, 3)
Test data:
(5000, 32, 32, 3)


In [9]:
num_classes = 10
img_h, img_w, img_c = X_train.shape[1:]

X_input = tf.placeholder(tf.float32, shape=[None, img_h, img_w, img_c])
Y_input = tf.placeholder(tf.float32, shape=[None, num_classes])

In [10]:
def build_model(X_inp, Y_inp):
    inp = X_inp
    output = Y_inp
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=inp,
        filters=16,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=conv1,
        filters=16,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)    

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #3
    conv3 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
        
    # Convolutional Layer #4
    conv4 = tf.layers.conv2d(
        inputs=conv3,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)    
    
    # Pooling Layer #1
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    # Dense layer:
    flat = tf.contrib.layers.flatten(pool2)
    logits = tf.layers.dense(inputs=flat, units=10, activation=None)
    
    return logits

In [11]:
logits = build_model(X_input, Y_input)
prediction = tf.nn.softmax(logits)


# Compute loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_input))

optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss=loss)

In [13]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y_input, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [16]:
init = tf.global_variables_initializer()

In [17]:
# tar = np.array(list(range(30)))

def get_next_batch(X, Y, step, batch_size):
    start = ((step - 1) * batch_size) % X.shape[0]
    end = start + batch_size
    return X[start:end], Y[start:end]

# for i in range(100):
#     print(get_next_batch(tar, i, 4))

In [19]:
batch_size = 32
epochs = 5
num_steps = epochs * X_train.shape[0] // batch_size
display_step = 20

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = get_next_batch(X_train, Y_train, step, batch_size)
#         print(batch_x)
#         print("="*20)
#         print(batch_y)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X_input: batch_x, Y_input: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            val_loss, acc = sess.run([loss, accuracy], feed_dict={X_input: X_val,
                                                                 Y_input: Y_val})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(val_loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X_input: X_test,
                                      Y_input: Y_test}))

Step 1, Minibatch Loss= 2.3003, Training Accuracy= 0.103
Step 20, Minibatch Loss= 2.2165, Training Accuracy= 0.192
Step 40, Minibatch Loss= 2.0586, Training Accuracy= 0.258
Step 60, Minibatch Loss= 2.0478, Training Accuracy= 0.259
Step 80, Minibatch Loss= 1.9771, Training Accuracy= 0.302
Step 100, Minibatch Loss= 1.9527, Training Accuracy= 0.308
Step 120, Minibatch Loss= 1.8222, Training Accuracy= 0.350
Step 140, Minibatch Loss= 1.7773, Training Accuracy= 0.365
Step 160, Minibatch Loss= 1.7843, Training Accuracy= 0.371
Step 180, Minibatch Loss= 1.7575, Training Accuracy= 0.367
Step 200, Minibatch Loss= 1.7206, Training Accuracy= 0.389
Step 220, Minibatch Loss= 1.7332, Training Accuracy= 0.366
Step 240, Minibatch Loss= 1.6795, Training Accuracy= 0.395
Step 260, Minibatch Loss= 1.6454, Training Accuracy= 0.424
Step 280, Minibatch Loss= 1.6104, Training Accuracy= 0.427
Step 300, Minibatch Loss= 1.6298, Training Accuracy= 0.415
Step 320, Minibatch Loss= 1.6729, Training Accuracy= 0.401
Ste

Step 2760, Minibatch Loss= 1.0064, Training Accuracy= 0.647
Step 2780, Minibatch Loss= 1.0592, Training Accuracy= 0.634
Step 2800, Minibatch Loss= 1.0420, Training Accuracy= 0.643
Step 2820, Minibatch Loss= 1.0248, Training Accuracy= 0.644
Step 2840, Minibatch Loss= 1.0118, Training Accuracy= 0.643
Step 2860, Minibatch Loss= 1.0772, Training Accuracy= 0.622
Step 2880, Minibatch Loss= 1.0026, Training Accuracy= 0.648
Step 2900, Minibatch Loss= 1.0488, Training Accuracy= 0.636
Step 2920, Minibatch Loss= 1.0217, Training Accuracy= 0.641
Step 2940, Minibatch Loss= 1.0070, Training Accuracy= 0.648
Step 2960, Minibatch Loss= 1.0056, Training Accuracy= 0.650
Step 2980, Minibatch Loss= 0.9825, Training Accuracy= 0.655
Step 3000, Minibatch Loss= 1.0776, Training Accuracy= 0.618
Step 3020, Minibatch Loss= 0.9783, Training Accuracy= 0.657
Step 3040, Minibatch Loss= 1.0056, Training Accuracy= 0.648
Step 3060, Minibatch Loss= 1.0890, Training Accuracy= 0.621
Step 3080, Minibatch Loss= 1.0008, Train

Step 5500, Minibatch Loss= 1.0154, Training Accuracy= 0.652
Step 5520, Minibatch Loss= 0.9212, Training Accuracy= 0.681
Step 5540, Minibatch Loss= 0.9042, Training Accuracy= 0.686
Step 5560, Minibatch Loss= 0.9450, Training Accuracy= 0.679
Step 5580, Minibatch Loss= 0.9178, Training Accuracy= 0.682
Step 5600, Minibatch Loss= 0.9206, Training Accuracy= 0.680
Step 5620, Minibatch Loss= 0.9522, Training Accuracy= 0.675
Step 5640, Minibatch Loss= 0.9579, Training Accuracy= 0.667
Step 5660, Minibatch Loss= 0.9051, Training Accuracy= 0.688
Step 5680, Minibatch Loss= 0.9667, Training Accuracy= 0.663
Step 5700, Minibatch Loss= 0.9215, Training Accuracy= 0.677
Step 5720, Minibatch Loss= 0.8892, Training Accuracy= 0.689
Step 5740, Minibatch Loss= 0.9040, Training Accuracy= 0.691
Step 5760, Minibatch Loss= 1.0023, Training Accuracy= 0.664
Step 5780, Minibatch Loss= 1.0071, Training Accuracy= 0.664
Step 5800, Minibatch Loss= 0.9026, Training Accuracy= 0.695
Step 5820, Minibatch Loss= 0.9366, Train